## Table of Contents

- [Predicting activity status: models & variables combinations to test](#predicting-activity-status-models--variables-combinations-to-test)
  - [Import libraries](#import-libraries)
  - [Set the directory](#set-the-directory)
  - [Import the final dataset](#import-the-final-dataset)
  - [Train vs. validation vs. test sets](#train-vs-validation-vs-test-sets)
  - [PCA biplot - representation of relationships observed between metrics (sample by boat trip) and models](#pca-biplot---representation-of-relationships-observed-between-metrics-sample-by-boat-trip-and-models)
  - [Calculating Global SHAP Feature Importance](#calculating-global-shap-feature-importance)
  - [Calculating Aggregated SHAP Feature Importance](#calculating-aggregated-shap-feature-importance)
  - [Generating the ROC curves](#generating-the-roc-curves)
  - [Mapping models' predictions of *fishing/not_fishing* for each trip](#mapping-models-predictions-of-fishingnotfishing-for-each-trip)

# Predicting activity status: models & variables combinations to test
Analysis of "1_predict_activity_status_parallel.py" output <br>

The following models were tested as classifiers, aiming at predicting the activity status: recognising if the fisher is fishing - hauling the gear - or not - setting the gear/navigation. <br>
1. **LoRe**: Logistic Regression;
2. **Dtree**: Decision Trees;
3. **RaFo**: Random Forests;
4. **XGBo**: Extreme Gradient Boosting. <br>

Different metrics were calculated to provide various perspectives on the performance of the tested models. <br>
1. **Accuracy**: the proportion of correctly classified instances (both positive and negative) out of the total number of instances.
2. **Precision**: the proportion of correctly predicted positive instances out of all instances predicted as positive.
3. **Recall** (also called sensitivity): the proportion of correctly predicted positive instances out of all actual positive instances.
4. **F1 score**: the harmonic mean of precision and recall.
5. **AUC score**:  the area under the Receiver Operating Characteristic (ROC) curve.
6. **Specificity**: the proportion of correctly predicted negative instances out of all actual negative instances. <br>

Furthermore, the same models and the same metrics have been tested and calculated with 7 variables, used as predictors to guide the models recognising the activity status.
* 7-variables: SPEED, course_diff, distance_from_coast (OR depth), hours, time_seconds, months, trip_duration  <br>
'hour' and 'months' were categorigal variables, one-hot encoded to make the models perceive them as factors.

## Import libraries

In [ ]:
import os
import numpy as np
import warnings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re
import ast
from sklearn.metrics import roc_curve, auc
import pyreadr

warnings.filterwarnings('ignore')

## Set the directory

In [ ]:
# Set directory paths
basedir = "path/to/your_folder/"  # must contain this script and the dataset
outdir = "results/"

## Import the final dataset

In [ ]:
# Load the output dataset from the Python script "1_predict_activity_status_parallel.py"
table = pd.read_csv(os.path.join(basedir,outdir, "model_performances_on_data_splitting_with_shap.csv"))
table['num_var'] = table['combo'].str.split(',').str.len()

## Train vs. validation vs. test sets

Let's plot the results recorded for the different calculated metrics, comparing performances among the three sets: train, validation and test. <br>

* **Train set** = **70%** of the fishing trips
* **Validation set** = **20%** of the fishing trips
* **Test set** = **10%** of the fishing trips (unseen data) <br>

Sample unit by boat should be used, as the sample unit by point can be too optimistic. Indeed, the latter uses random points from different trips to train the models instead of using the full trips, which is unrealistic for the type of data that we are working with (time-series), as stated by Samarão *et al.*, 2024. <br>

The following plot shows performances of the 6 listed evaluation metrics for the 4 aforementioned classification models, with data splitting into train, validation and test sets. Boxplots display the distribution of training (blue) and validation (green) scores across cross-validation folds. The blue and green numeric labels indicate the mean score for their respective sets. The red diamond and its red numeric label show the final, single score achieved on the independent test set. A 90% threshold (red dashed line) is included for reference. 

In [ ]:
summary_data = pd.DataFrame()
metrics = ['accuracy', 'precision', 'recall','f1', 'specificity', 'auc']

# Reshape and compute summary statistics
for metric in metrics:
    temp = table[['model', 'num_var', f'{metric}_train', f'{metric}_val', f'{metric}_test']].copy()
    temp = temp.melt(id_vars=['model', 'num_var'],
                     value_vars=[f'{metric}_train', f'{metric}_val', f'{metric}_test'],
                     var_name='set',
                     value_name='value')
    temp['metric'] = metric
    temp['set'] = temp['set'].str.extract(r'_(train|val|test)')
    summary_data = pd.concat([summary_data, temp], ignore_index=True)

# Filter out empty cells
summary_data = summary_data[summary_data['value'].notna()]

def get_whisker_max(data):
    """Calculates the maximum value displayed by the upper whisker (Q3 + 1.5*IQR)."""
    if data.empty:
        return np.nan
    q3 = data.quantile(0.75)
    iqr = q3 - data.quantile(0.25)
    # Max value is the maximum data point that is <= (Q3 + 1.5 * IQR)
    whisker_max = data[data <= (q3 + 1.5 * iqr)].max()
    # Handle cases where all data might be outliers, return Q3 in a pinch
    return whisker_max if not pd.isna(whisker_max) else q3

set_order = ['train', 'val', 'test']
summary_data['set'] = pd.Categorical(summary_data['set'], categories=set_order, ordered=True)

model_order = ['LoRe', 'Dtree', 'RaFo', 'XGBo']
metric_order = ['accuracy', 'precision', 'recall', 'f1','specificity', 'auc']
summary_data['model'] = pd.Categorical(summary_data['model'], categories=model_order, ordered=True)
summary_data['metric'] = pd.Categorical(summary_data['metric'], categories=metric_order, ordered=True)

metric_title_map = {m: m.capitalize() for m in metric_order}
summary_data['metric_capitalized'] = summary_data['metric'].map(metric_title_map)
summary_data['metric_capitalized'] = pd.Categorical(
    summary_data['metric_capitalized'],
    categories=[metric_title_map[m] for m in metric_order],
    ordered=True
)

boxplot_data = summary_data[summary_data['set'].isin(['train', 'val'])].copy()
point_data = summary_data[summary_data['set'] == 'test'].copy()
mean_data = boxplot_data.groupby(['model', 'metric', 'set'])['value'].mean().reset_index()


# PERSONALIZED OFFSETS --------------------------------
# We use a single offset for train/val/test, applied to the calculated Y position
label_offset = 0.005 
label_format = "{:.2f}" 
# Offsets for X-axis positioning
x_offset_standard = 0.25
x_pos_train = -x_offset_standard      
x_pos_test = x_offset_standard        
x_pos_val = 0.00 


# PLOT GENERATION: USING PLT.SUBPLOTS ----------------
sns.set(style="whitegrid", font_scale=1.2)

n_metrics = len(metric_order)
n_cols = 6
n_rows = 1 
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 6, 7), sharey=False) 

axes = axes.flatten()

box_handles = []
box_labels = []

for i, metric in enumerate(metric_order):
    ax = axes[i]
    metric_title = metric_title_map[metric]
    
    metric_data_box = boxplot_data[boxplot_data['metric'] == metric]
    metric_data_mean = mean_data[mean_data['metric'] == metric]
    metric_data_test = point_data[point_data['metric'] == metric]

    # Plot the Boxplot 
    legend_status = True if i == 0 else False
    sns.boxplot(data=metric_data_box, x='model', y='value', hue='set',
                palette='viridis', showfliers=False, width=0.7, ax=ax,
                boxprops=dict(alpha=0.8),
                medianprops=dict(color='black', linewidth=2),
                legend=legend_status)

    # Add Numeric Labels (Test and Mean for Train/Val)
    for j, model in enumerate(model_order):
        
        # --- TEST SET VALUE ---
        model_point = metric_data_test[metric_data_test['model'] == model]
        if len(model_point) == 1:
            test_value = model_point['value'].iloc[0]

            ax.scatter(j + x_pos_test, test_value, color='red', marker='D', s=70,
                        edgecolors='black', linewidths=1,
                        label='test' if (i == 0 and j == 0) else None) 
            
            # Position is test_value + offset
            ax.text(j + x_pos_test, test_value + label_offset,
                        label_format.format(test_value),
                        color='red', ha='center', va='bottom', fontsize=10, weight='bold')

        # --- TRAIN SET MEAN LABEL (Overlap prevention) ---
        mean_train_df = metric_data_mean[(metric_data_mean['model'] == model) & (mean_data['set'] == 'train')]
        train_full_data = summary_data[(summary_data['model'] == model) & (summary_data['metric'] == metric) & (summary_data['set'] == 'train')]['value']
        
        if not mean_train_df.empty:
            mean_train = mean_train_df['value'].iloc[0]
            whisker_max = get_whisker_max(train_full_data)
            
            # The Y position is the maximum of the mean or the whisker max, plus offset
            y_pos_train = max(mean_train, whisker_max) + label_offset

            ax.text(j + x_pos_train, y_pos_train,
                        label_format.format(mean_train), # We still plot the mean value
                        color='blue', ha='center', va='bottom', fontsize=9)

        # --- VALIDATION SET MEAN LABEL (Overlap prevention) ---
        mean_val_df = metric_data_mean[(metric_data_mean['model'] == model) & (mean_data['set'] == 'val')]
        val_full_data = summary_data[(summary_data['model'] == model) & (summary_data['metric'] == metric) & (summary_data['set'] == 'val')]['value']
        
        if not mean_val_df.empty:
            mean_val = mean_val_df['value'].iloc[0]
            whisker_max = get_whisker_max(val_full_data)
            
            # The Y position is the maximum of the mean or the whisker max, plus offset
            y_pos_val = max(mean_val, whisker_max) + label_offset

            ax.text(j + x_pos_val, y_pos_val,
                        label_format.format(mean_val),
                        color='green', ha='center', va='bottom', fontsize=9)


    # Final Axis Styling
    ax.set_ylim(0.8, 1)
    ax.set_yticks(np.arange(0.75, 1.01, 0.05))
    ax.set_xlabel("") 
    ax.tick_params(axis='x', rotation=45)
    ax.axhline(0.9, color='red', linestyle='--', linewidth=1) 
    ax.set_ylabel(metric_title, fontsize=14, labelpad=10, weight='bold') 
    
    if i > 0:
        ax.tick_params(axis='y', labelleft=False)
    if i == 0:
        box_handles, box_labels = ax.get_legend_handles_labels()
        if ax.legend_ is not None:
            ax.legend_.remove()
    elif ax.legend_ is not None:
        ax.legend_.remove() 


# Global Legend --------------------------------------
cleaned_box_handles = [h for h, l in zip(box_handles, box_labels) if l in ['train', 'val']]
cleaned_box_labels = [l for l in box_labels if l in ['train', 'val']]

test_handle_fixed = plt.Line2D([], [], 
                               color='red', 
                               marker='D',          
                               linestyle='None', 
                               markersize=8, 
                               label='test', 
                               markerfacecolor='red', 
                               markeredgecolor='black', 
                               markeredgewidth=1)      

final_handles = cleaned_box_handles + [test_handle_fixed]
final_labels = cleaned_box_labels + ['test']

fig.legend(title="Dataset Split", handles=final_handles, labels=final_labels,
           bbox_to_anchor=(1.02, 0.5), loc='center left', borderaxespad=0.)

# Final Layout and Save
plt.tight_layout(rect=[0, 0, 0.95, 1])
filename = "hauls_multi_class_performance_boxplots_with_test.png"
full_path = os.path.join(basedir, outdir, filename)
plt.savefig(full_path, dpi=300, bbox_inches='tight')
plt.show()

## Principal Components Analysis (PCA) on the test set

This chunk performs a Principal Component Analysis (PCA) on standardized performance metrics (accuracy, precision, recall, F1-score, specificity, and AUC) computed on the final test set. After removing incomplete cases, the metrics are scaled and used to derive principal components. 
* A **PCA biplot** is produced to visualize and compare model performance in the space of the first two components, showing both model scores and variable loadings. 
* In addition, a **scree plot** and **cumulative explained variance plot** are generated to assess the contribution of each component and determine how much variance is captured by the leading components.

In [ ]:
results_df = table.copy()
test_results_df = results_df[results_df['fold'] == 'final_test'].copy()
metrics_for_pca = ['accuracy_test', 'precision_test', 'recall_test','f1_test', 'specificity_test', 'auc_test']

test_results_pca = test_results_df.dropna(subset=metrics_for_pca).copy()

if test_results_pca.empty:
    print("No valid test set metrics found for PCA.")
else:
    # PCA Calculation
    X = test_results_pca[metrics_for_pca]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    pca = PCA()
    principal_components = pca.fit_transform(X_scaled)
    
    pca_df = pd.DataFrame(
        data=principal_components[:, :2], 
        columns=['PC1', 'PC2']
    )
    pca_df = pd.concat([test_results_pca[['model']].reset_index(drop=True), pca_df], axis=1)

    ### 1. PCA biplot ###
    plt.figure(figsize=(9, 7))
    sns.scatterplot(x='PC1', y='PC2', hue='model', data=pca_df, s=200, marker='o', zorder=3)

    # Add Scaled Arrows - adjusts arrow length to match the scale of the PC scores 
    loadings = pca.components_
    scale_factor = np.max(np.abs(principal_components[:, :2])) / np.max(np.abs(loadings[:2, :])) * 0.8

    for i, feature in enumerate(metrics_for_pca):
        x_arr = loadings[0, i] * scale_factor
        y_arr = loadings[1, i] * scale_factor
        plt.arrow(0, 0, x_arr, y_arr, color='r', alpha=0.5, linewidth=1.5, head_width=0.1, head_length=0.15)
        plt.text(x_arr * 1.1, y_arr * 1.1, feature, color='r', alpha=0.8, fontsize=11, fontweight='bold')

    plt.axhline(y=0, color='grey', linewidth=1, linestyle='--')
    plt.axvline(x=0, color='grey', linewidth=1, linestyle='--')
    plt.title(f'PCA Biplot: Model Performance (Test Set)\nPC1: {pca.explained_variance_ratio_[0]:.1%} | PC2: {pca.explained_variance_ratio_[1]:.1%}')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.grid(True, alpha=0.3)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    filename = "PCA_biplot.png"
    full_path = os.path.join(basedir, outdir, filename)
    plt.savefig(full_path, dpi=300, bbox_inches='tight')
    plt.show()

    ### 2. Scree Plot & Cumulative Variance ###
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Scree Plot (Eigenvalues)
    eigenvalues = pca.explained_variance_
    ax1.plot(range(1, len(eigenvalues) + 1), eigenvalues, marker='o', color='steelblue', linewidth=2)
    ax1.axhline(y=1, color='r', linestyle='--', label='Kaiser Criterion (λ=1)')
    ax1.set_title('Scree Plot (Importance of Components)')
    ax1.set_xlabel('Principal Component')
    ax1.set_ylabel('Eigenvalue')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Cumulative Explained Variance
    cum_variance = np.cumsum(pca.explained_variance_ratio_)
    ax2.plot(range(1, len(cum_variance) + 1), cum_variance, marker='o', color='seagreen', linewidth=2)
    ax2.axhline(y=0.95, color='orange', linestyle=':', label='95% Threshold')
    ax2.set_title('Cumulative Explained Variance')
    ax2.set_xlabel('Number of Components')
    ax2.set_ylabel('Total Variance Explained (%)')
    ax2.set_ylim(0, 1.05)
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    filename = "Scree_plot_and_cumulative_variance.png"
    full_path = os.path.join(basedir, outdir, filename)
    plt.savefig(full_path, dpi=300, bbox_inches='tight')
    plt.show()
    plt.show()

    print(f"Variance explained by PC1 + PC2: {cum_variance[1]:.2%}")

## Calculating Global SHAP Feature Importance
This code generates global SHAP feature importance visualizations for each model evaluated on the final test set. Rows corresponding to the final test fold with available SHAP information are selected, and the stored feature importance values are parsed from their dictionary representation. For each model, features are ranked by their mean absolute SHAP values and displayed in a bar chart, highlighting the relative contribution of each feature to the model’s predictions. <br>
Here are shown all the levels of *hours* and *months* categorical variables.

In [ ]:
df = table.copy()

# Filter for final test rows where SHAP data exists
final_test_df = df[df['fold'] == 'final_test'].dropna(subset=['feature_importance']).copy()

for idx, row in final_test_df.iterrows():
    model_name = row['model']
    
    try:
        # The 'feature_importance' column is a string representation of a dict
        # Use ast.literal_eval to safely convert it to a real Python dictionary
        imp_dict = ast.literal_eval(row['feature_importance'])
        
        # Sort features by importance value
        sorted_imp = sorted(imp_dict.items(), key=lambda x: x[1], reverse=True)
        features, values = zip(*sorted_imp)

        plt.figure(figsize=(10, 8))
        sns.barplot(x=list(values), y=list(features), palette="viridis")
        plt.title(f'SHAP Global Feature Importance: {model_name}', fontsize=14)
        plt.xlabel('Mean |SHAP value| (Average Impact on Model Output)')
        plt.ylabel('Features')
        plt.tight_layout()
        
        save_path = os.path.join(basedir, outdir, f'shap_importance_{model_name}.png')
        plt.savefig(save_path, dpi=300)
        plt.close()
        print(f"Created shap_importance_{model_name}.png")
        
    except Exception as e:
        print(f"Could not parse importance for {model_name}: {e}")

print("\nDone! All plots are in the 'plots' folder.")

## Calculating Aggregated SHAP Feature Importance 
This code computes and visualizes aggregated global SHAP feature importance for models evaluated on the final test set. SHAP values are parsed from their stored dictionary format and categorical features (*hours* and *months*), are collapsed by summing their respective importance values across levels. All remaining features are retained individually. The aggregated and original features are then ranked by mean absolute SHAP value and displayed in bar plots, providing a more interpretable summary of the relative influence of temporal and continuous predictors.

In [ ]:
final_test_df = df[df['fold'] == 'final_test'].dropna(subset=['feature_importance']).copy()

for idx, row in final_test_df.iterrows():
    model_name = row['model']
    
    try:
        raw_imp_dict = ast.literal_eval(row['feature_importance'])
        
        clean_imp_dict = {}
        hours_total = 0.0
        month_total = 0.0
        
        # Aggregate levels for categorical variables (hours and months)
        for k, v in raw_imp_dict.items():
            if k.startswith('hours_'):
                hours_total += v
            elif k.startswith('months_'):
                month_total += v
            else:
                # Keep other variables (SPEED, distance_to_coast, etc.) as they are
                clean_imp_dict[k] = v
        
        # Add the aggregated totals with your requested naming convention
        clean_imp_dict['hours (aggregated)'] = hours_total
        clean_imp_dict['months (aggregated)'] = month_total

        # Sort features by importance value
        sorted_imp = sorted(clean_imp_dict.items(), key=lambda x: x[1], reverse=True)
        features, values = zip(*sorted_imp)

        # Plotting
        plt.figure(figsize=(10, 8))
        sns.barplot(x=list(values), y=list(features), palette="viridis")
        
        plt.title(f'Aggregated SHAP Feature Importance: {model_name}', fontsize=14)
        plt.xlabel('Mean |SHAP value| (Average Impact on Model Output)')
        plt.ylabel('Features')
        
        plt.tight_layout()
        save_path = os.path.join(basedir,outdir, f'shap_importance_{model_name}_aggregated.png')
        plt.savefig(save_path, dpi=300)
        plt.close()
        print(f"Created aggregated plot for {model_name}")
        
    except Exception as e:
        print(f"Error processing {model_name}: {e}")

## Generating the ROC curves

The ROC curve plots the True Positive Rate (Recall) against the False Positive Rate (FPR) at various threshold settings. The AUC provides a single scalar value that summarizes the overall performance of a binary classifier across all possible classification thresholds. <br>
AUC = 1: The model perfectly distinguishes between all positive and negative instances. <br>
AUC = 0.5: The model's performance is no better than random guessing. <br>
AUC < 0.5: The model is performing worse than random guessing (this usually indicates an issue with the model). <br>
Generally, the higher the AUC, the better the model's ability to discriminate between positive and negative classes. <br>

ROC curves are essential because they measure a model's ability to distinguish between classes across all possible decision thresholds, rather than relying on a single fixed point. This makes them highly effective for tracking data where fishing events are often less frequent than transit. By mapping the trade-off between sensitivity and false alarms, they provide an objective Area Under the Curve (AUC) score that validates a model’s reliability. High AUC values demonstrate that the model has successfully identified complex behavioral patterns rather than biased trends.

In [ ]:
#  Load the output file from the Python script "1_predict_activity_status_parallel.py"
predictions = pyreadr.read_r(os.path.join(basedir, outdir, 'dataset_with_predictions.rds'))
df = predictions[None] 

test_df = df[df['set'] == 'test'].copy() # we need just the predictions on unseen data

# Define Confusion Categories mapping
def get_confusion_cat(row, model_name):
    target = row['target']
    pred = row[f'target_pred_{model_name}']
    if target == 0 and pred == 0: return 'TP'
    if target == 1 and pred == 0: return 'FP'
    if target == 1 and pred == 1: return 'TN'
    if target == 0 and pred == 1: return 'FN'
    return 'None'

models =  ['LoRe', 'Dtree', 'RaFo', 'XGBo']
for model in models:
    test_df[f'cat_{model}'] = test_df.apply(lambda row: get_confusion_cat(row, model), axis=1)

# Save the categorized test set
save_path_csv = os.path.join(basedir, outdir, 'test_set_with_confusion_categories.csv')
test_df.to_csv(save_path_csv, index=False)

# Plot ROC Curves
plt.figure(figsize=(8, 6))
for model in models:
    # Convert predictions and probabilities to numeric to avoid "int and str" errors
    test_df[f'target_pred_{model}'] = pd.to_numeric(test_df[f'target_pred_{model}'], errors='coerce')
    test_df[f'target_prob_{model}'] = pd.to_numeric(test_df[f'target_prob_{model}'], errors='coerce')
    # We use 1 - prob because prob_MODEL is P(target=1), but Fishing is 0.
    y_true = (test_df['target'] == 0).astype(int) 
    y_score = 1 - test_df[f'target_prob_{model}'] 
    
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{model} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel('False Positive Rate (Fishing)')
plt.ylabel('True Positive Rate (Fishing)')
plt.title('ROC Curves - Fishing Activity Prediction')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
save_path_roc = os.path.join(basedir, outdir, "roc_curves.png")
plt.savefig(save_path_roc)
plt.close()

## Mapping models' predictions of *fishing/not_fishing* for each trip

The present chunk is useful for generating spatial maps for each test trip, visualizing model predictions along vessel trajectories. For every unique trip, geographic points are plotted in longitude–latitude space and colored according to classification outcomes (TP,FP,TN,FN). Separate panels are created for each model to enable direct comparison of spatial prediction patterns within the same trip. All trip-level maps are saved to disk, providing a visual assessment of where and how models succeed or fail in space. <br>

Remember that:
* TP = True Positives (correctly predicted positive)
* TN = True Negatives (correctly predicted negative)
* FP = False Positives (incorrectly predicted positive)
* FN = False Negatives (incorrectly predicted negative) <br>

Note that, in our case, TP are the fishing points (target == 0), while non-fishing points are TN (target == 1). <br>
In the generated maps, TP/FP/TN/FN are highlighted in different colours.

In [ ]:
trip_maps_path = os.path.join(basedir, outdir, 'trip_maps')
if not os.path.exists(trip_maps_path):
    os.makedirs(trip_maps_path)

def save_all_trip_maps(df, models, output_dir):
    test_trip_ids = df['boat_trip_id'].unique()
    colors = {'TP': '#2ca02c', 'TN': '#1f77b4', 'FP': '#d62728', 'FN': '#ff7f0e'}
    
    print(f"Generating {len(test_trip_ids)} trip maps...")
    
    for trip_id in test_trip_ids:
        trip_data = df[df['boat_trip_id'] == trip_id].sort_values('DATE_TIME')
        
        fig, axes = plt.subplots(1, len(models), figsize=(20, 8), sharex=True, sharey=True)
        if len(models) == 1: axes = [axes]
        
        for ax, model in zip(axes, models):
            for cat, color in colors.items():
                cat_data = trip_data[trip_data[f'cat_{model}'] == cat]
                if not cat_data.empty:
                    ax.scatter(cat_data['longitude'], cat_data['latitude'], 
                               c=color, label=cat, s=15, alpha=0.6)
            
            ax.set_title(f'Trip {trip_id} Performance: {model}')
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')
            ax.legend(loc='best', markerscale=2)
        
        plt.tight_layout()
        save_path_map = os.path.join(output_dir, f'map_trip_{trip_id}.png')
        plt.savefig(save_path_map)
        plt.close()

save_all_trip_maps(test_df, models, trip_maps_path)

print(f"✅ Analysis complete. ROC saved to {save_path_roc} and {len(test_df['boat_trip_id'].unique())} maps saved to {trip_maps_path}")